In [ ]:
from keras.utils import Sequence
from skimage.io import imread
from skimage.transform import resize
import numpy as np

class MY_Generator(Sequence):

    def __init__(self, image_filenames, labels, batch_size):
        self.image_filenames, self.labels = image_filenames, labels
        self.batch_size = batch_size

    def __len__(self):
        return np.ceil(len(self.image_filenames) / float(self.batch_size))

    def __getitem__(self, idx):
        batch_x = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        return np.array([
            resize(imread(file_name), (28, 28))
               for file_name in batch_x]), np.array(batch_y)

In [ ]:
from keras.utils import Sequence
class MY_Generator(Sequence):

    def __init__(self, image_filenames, labels, batch_size):
        self.image_filenames, self.labels = image_filenames, labels
        self.batch_size = batch_size

    def __len__(self):
        return np.ceil(len(self.image_filenames) / float(self.batch_size))

    def __getitem__(self, idx):
        batch_x = self.image_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]

        return np.array([
            Image.open(file_name).resize([28,28])
               for file_name in batch_x]), np.array(batch_y)

In [ ]:
im = Image.open(str(files[1])).resize([28,28])

In [1]:
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers
from pathlib import Path
import keras
import numpy as np
from PIL import Image

p = Path('./data/') 
files = list(p.glob('**/*.png'))
files = str(files).split("'")
files = [ files[i] for i in list(range(1,len(files),2)) ]
classes = str(files).split('/')
classes = [ classes[i] for i in list(range(1,len(classes),2)) ]
classnames, indices = np.unique(classes, return_inverse=True)
labels = keras.utils.to_categorical(indices, len(np.unique(indices)))

Using TensorFlow backend.


In [4]:
im=Image.open(files[2]).resize([28,28])
np.array(im)

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [5]:
batch_size = 10
num_epochs = 1
activation = "relu"
LR = 0.01
L2 = 0.001
num_training_samples = 106

my_training_batch_generator = MY_Generator(files, labels, batch_size)
#my_validation_batch_generator = My_Generator(validation_filenames, GT_validation, batch_size)

model = Sequential([
    Flatten(input_shape = (28, 28, 3), name = "flatten"),
    Dense(50, name = "hidden_1", kernel_regularizer=regularizers.l2(L2),),
    Activation(activation, name = "act_hidden_1"),
    Dense(10, name = "out"),
    Activation("softmax", name = "act_out")
    ])

sgd = optimizers.SGD(lr=LR, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer = sgd,
             loss = "categorical_crossentropy",
             metrics = ["accuracy"])

model.fit_generator(generator=my_training_batch_generator,
                                          steps_per_epoch=(num_training_samples // batch_size),
                                          epochs=num_epochs,
                                          verbose=1,
                                          #validation_data=my_validation_batch_generator,
                                          #validation_steps=(num_validation_samples // batch_size),
                                          use_multiprocessing=True,
                                          workers=16,
                                          max_queue_size=32)

NameError: name 'MY_Generator' is not defined

In [ ]:
def simple_NN(x_train, y_train, x_valid, y_valid, L2 = 0.001, activation = "relu", LR = 0.01, batch_size = 64):
    from keras import optimizers
    model = Sequential([
        Flatten(input_shape = (1,28, 28), name = "flatten"),
        Dense(50, name = "hidden_1", kernel_regularizer=regularizers.l2(L2),),
        Activation(activation, name = "act_hidden_1"),
        Dense(10, name = "out"),
        Activation("softmax", name = "act_out")
    ])
    sgd = optimizers.SGD(lr=LR, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer = sgd,
             loss = "categorical_crossentropy",
             metrics = ["accuracy"])
    checkpoints = keras.callbacks.ModelCheckpoint("./checkpoints/model_{epoch:02d}.hdf5", 
                                              monitor='val_loss', 
                                              verbose=0, 
                                              save_best_only=False, save_weights_only=False)
    history = model.fit(x_train, y_train, 
                    epochs = 10, batch_size = batch_size, 
                    callbacks = [checkpoints], 
                    validation_data = (x_valid, y_valid))
    return model, history

In [ ]:
from pathlib import Path
p = Path('./data/imgs/') 
classes = [x for x in p.iterdir() if x.is_dir()]
files = list(p.glob('**/*.jpg'))

In [ ]:
str(files[200000])

In [ ]:
# euc laptop
from pathlib import Path
p = Path('./data/') 
#classes = [x for x in p.iterdir() if x.is_dir()]
files = list(p.glob('**/*.png'))
classes = str(files).split('/')
classes = [ classes[i] for i in list(range(1,len(classes),2)) ]

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(plt.imread(str(files[1])))
plt.show()

In [ ]:
from PIL import Image
im = Image.open(str(files[1])).resize([28,28])
#im = im.resize([28,50])
plt.imshow(im)
plt.show()

In [ ]:
import numpy as np
im = np.asarray(im)
im.shape

In [ ]:
import keras
keras.utils.to_categorical(classes, len(np.unique(classes)))

In [ ]:
classnames, indices = np.unique(classes, return_inverse=True)
keras.utils.to_categorical(indices, len(np.unique(indices)))

In [ ]:
files2[3]